# All filters generator

### Parameters

In [1]:
N = 16      # Number of pixels by block
FRAC = 4    # Number of fractional pixels for interpolation (eg: 1/4 of resolution)
MODE = 0    # Select MC(0) or FME(1):
SEARCH = 2  # [Only for MC] Select Horizontal search(0), Vertical search(1) or Full search(2)
TAPS = 8    # Number of taps in the filter
I = 8       # Input size of the line 
S = 1       # Stride=1 means that the filter will deslocate 1 by 1 pixel at the time

In [2]:
import math
import numpy as np
from matplotlib import pyplot as plt

### Final Verilog text

In [4]:
verilog = '''/*------------------------------------------------------------------------------
 * File: all_filters.v
 * Date generated: 13/02/2023
 * Date modified: 13/02/2023
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters
 *------------------------------------------------------------------------------ */

module all_filters (
    X,
    Y1
);


// ------------------------------------------
// Parameters
// ------------------------------------------

    // Bit width of input data 
    parameter integer IN_SIZE = 'd8;
	 
	 
// ------------------------------------------
// Port mode declarations
// ------------------------------------------

//  input  signed [IN_SIZE-1:0] X;
//  output signed [9:0]   	Y1;
//  output signed [9:0]    	Y2

// ------------------------------------------
// Wires declarations
// ------------------------------------------

//  wire signed [8:0] w2;
//  wire signed [9:0] w2_;

	 
// ------------------------------------------
// Combinational logic
// ------------------------------------------

//  assign w1 = X;
//  assign w4 = w1 << 2;
//  assign w3 = w4 - w1;
//  assign w5 = w1 + w4;
//  assign w2_ = -1 * w2;

  
// ------------------------------------------
// Outputs
// ------------------------------------------

//  assign Y1 = w2_;


endmodule //all_filters

'''
print(verilog)

/*------------------------------------------------------------------------------
 * File: all_filters.v
 * Date generated: 13/02/2023
 * Date modified: 13/02/2023
 * Author: Bruna Suemi Nagai
 * Description: Concatenating all 15 filters
 *------------------------------------------------------------------------------ */

module all_filters (
    X,
    Y1
);


// ------------------------------------------
// Parameters
// ------------------------------------------

    // Bit width of input data 
    parameter integer IN_SIZE = 'd8;
	 
	 
// ------------------------------------------
// Port mode declarations
// ------------------------------------------

//  input  signed [IN_SIZE-1:0] X;
//  output signed [9:0]   	Y1;
//  output signed [9:0]    	Y2

// ------------------------------------------
// Wires declarations
// ------------------------------------------

//  wire signed [8:0] w2;
//  wire signed [9:0] w2_;

	 
// ------------------------------------------
// Combinational logi